In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import json
import math
import pickle
import urllib.request
import dateutil.parser
import dateutil.rrule
import dateutil.tz
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sys
import gc
import re
from IPython.display import display, HTML
from textwrap import wrap

matplotlib.rcParams.update({
    'font.size': 13,
    'timezone': 'Europe/London'
})

## Each camera during the last 28 days

In [ ]:
# Used across most of the plots for people flows
# Used across most of the plots for people flows
tzLocal = dateutil.tz.gettz('Europe/London')
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time()).replace(tzinfo=tzLocal)
peopleCountInterval = 900
peopleCountFrames = pickle.load(open('../cache/recent-pedestrian-flows-pd.pkl', 'rb'))

# TODO: Make this reflect the last entry in the frame, not the time now
print('Last data obtained %s' 
    % (np.max(list(map(lambda f: np.max(f.index), peopleCountFrames.values()))).strftime('%d %B %Y %H:%M')))

In [ ]:
colourUp = '#f64a8a'
colourDown = '#233067'
# Ignore non-numeric columns in the dataframe
plottableTypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
perMinuteFactor = (peopleCountInterval / 60)

In [ ]:
historicCutOffDays = 28

for sensorName in sorted(peopleCountFrames.keys()):
    dfSensor = peopleCountFrames[sensorName]
    dfSensor = dfSensor \
        [dfSensor.index >= dateToday - pd.Timedelta(days=historicCutOffDays)] \
        .copy()

    # We may have data from the API, but nothing in the last N days, so don't attempt
    # a graph
    if np.all(np.isnan(dfSensor)):
        continue

    # Show the last two days of data as a larger area, so the detail is visible
    axisBreak = datetime.datetime.now(tzLocal) - datetime.timedelta(days=2)

    # People per minute for 'walking' columns
    directionDataOld = [dfSensor[ : axisBreak][direction] / perMinuteFactor
        for direction in dfSensor.select_dtypes(plottableTypes).columns]
    directionDataRecent = [dfSensor[axisBreak : ][direction] / perMinuteFactor
        for direction in dfSensor.select_dtypes(plottableTypes).columns]
    directionLabels = [direction for direction in dfSensor.columns]

    fig = plt.figure(figsize=(18,6.5))
    gs = fig.add_gridspec(ncols=2, nrows=1, width_ratios=[3, 1])
    ax = fig.add_subplot(gs[0, 0])
    axRecent = fig.add_subplot(gs[0, 1])

    fig.suptitle(sensorName)
    ax.set_xlabel('Date')
    axRecent.set_xlabel('Last 48 hours')
    ax.set_ylabel('Pedestrians per minute')

    for i in range(len(directionDataOld)):
        ax.stackplot(
          dfSensor[ : axisBreak].index, 
          np.zeros(len(directionDataOld[i])),
          directionDataOld[i] * (-1 if i % 2 == 0 else +1),
          colors=[colourUp] if i % 2 == 0 else [colourDown]
        )
        axRecent.stackplot(
          dfSensor[axisBreak : ].index, 
          np.zeros(len(directionDataRecent[i])),
          directionDataRecent[i] * (-1 if i % 2 == 0 else +1), 
          labels=[directionLabels[i]],
          colors=[colourUp] if i % 2 == 0 else [colourDown]
        )

    yMax = math.ceil(np.max(np.amax(dfSensor.select_dtypes(plottableTypes) / perMinuteFactor, axis=None)))

    if np.isnan(yMax):
        continue

    ax.set_xlim([np.min(dfSensor.index), axisBreak])
    ax.set_ylim([-yMax, yMax])
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, pos: abs(y)))
    axRecent.set_xlim([axisBreak, np.max(dfSensor.index)])
    axRecent.set_ylim([-yMax, yMax])
    axRecent.yaxis.tick_right()
    axRecent.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, pos: abs(y)))

    dataFormatMajor = mdates.DateFormatter('%a %d %b')
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1, byweekday=mdates.MO))
    ax.xaxis.set_major_formatter(dataFormatMajor)
    ax.xaxis.set_tick_params(which='major', pad=15)
    ax.margins(x=0)
    axRecent.xaxis.set_major_locator(mdates.DayLocator(interval=1))
    axRecent.xaxis.set_major_formatter(mdates.DateFormatter('%H\n%d'))
    axRecent.margins(x=0)

    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
    axRecent.xaxis.set_minor_locator(mdates.HourLocator(byhour=[0, 6, 12, 18]))
    axRecent.xaxis.set_minor_formatter(mdates.DateFormatter('%H'))

    # Highlight areas with missing data or zero people counts
    #dfGaps = np.any(dfSensor.resample('3600s').sum() == 0.0, axis=1)
    #ax.fill_between(dfGaps.index, 0, ax.get_ylim()[1], where=dfGaps,
    #            facecolor='grey', alpha=0.1)

    ax.spines['right'].set_visible(False)
    axRecent.spines['left'].set_visible(False)

    fig.subplots_adjust(wspace=0.04)
    axRecent.legend()
    
    plt.savefig(
        '../output/pedestrian-flows_profile-28-days_%s.png' 
        % re.compile('[^a-z0-9]+').sub('-', sensorName.lower())
    )
